## 주가 관련 기사 크롤링

In [268]:
from ssl import SSLError
import pandas as pd
import datetime
from urllib import parse
from urllib.error import URLError
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import socket
import random
import argparse
import datetime
import pandas as pd
import requests
import nltk
import warnings
import random
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [269]:
def crawl(query, save_as, begin, end, sort=0, field=1, delay=0.5, timeout=30, page_limit=50):
    '''
    :param query: 네이버 '뉴스'란에서 검색할 검색어
    :param save_as: 검색 결과 저장 경로
    :param begin: '기간' -> 검색 기간 시작
    :param end: '기간' -> 검색 기간 끝
    :param sort: '유형' -> 0(관련도순) 1(최신순) 2(오래된순)
    :param field: '영역' -> 0(전체) 1(제목)
    :param delay: (옵션) 검색 리퀘스트 간격 (초)
    :param timeout: (옵션) 타임아웃 시 기다릴 시간 (초)
    :param page_limit: (옵션) 검색 결과에서 몇 페이지까지 갈 것인지 결정
    :return:
    '''

    # prerequisite
    df = pd.DataFrame(columns=['link', 'title', 'date', 'article'])

    # index settings
    # a single pages includes 10 news, starting from page 1 (index 1~10)
    current_index = 1
    max_index = 2

    while (current_index <= max_index) and (1 + current_index // 10 <= page_limit):
        url = make_url(query, sort, field, begin, end, current_index)
        bsobj = make_bsobj(url, delay, timeout, trial=10)
        if bsobj is None:
            continue
        naver_news_urls = make_naver_news_urls(bsobj)
        naver_news_title = get_naver_news_title(bsobj)
        try:
            img_url = get_article_img_url(bsobj)
        except:
            img_url = " "
        naver_news_articles = []
        try:
            if len(naver_news_urls) != 0:
                url = naver_news_urls[0]
                news_bsobj = BeautifulSoup(url, 'lxml')

                naver_news_article = get_naver_news_article_one(bsobj)
                naver_news_articles.append(naver_news_article[0])
            else:
                naver_news_title = " "
                naver_news_articles = " "
                img_url = " "

    #             date, article, title, newspaper = attributes
            df = pd.DataFrame([ x for x in zip(naver_news_title, naver_news_articles, img_url)])
            df.columns = ['naver_news_title', 'naver_news_articles', 'naver_news_img_url']
            current_index += 10

            max_index = get_max_index(bsobj)
            return df
        except:
            naver_news_title = " "
            naver_news_articles = " "
            img_url = " "
            df = pd.DataFrame([ x for x in zip(naver_news_title, naver_news_articles, img_url)])
            df.columns = ['naver_news_title', 'naver_news_articles', 'naver_news_img_url']
            return df

In [282]:
def get_max_index(bsobj):
    paging = bsobj.find_all('a', 'btn_next')
    if not paging:
        print('(WARNING!) no results found')
        return None
    return True

In [283]:
def make_naver_news_urls(bsobj):
    return [link['href'] for link in bsobj.find_all('a', 'news_tit')]

def get_naver_news_title(bsobj):
    return [link.text for link in bsobj.find_all('a', 'news_tit')]

def get_naver_news_article_one(bsobj):
    return [article.text for article in bsobj.find_all('a', 'api_txt_lines dsc_txt_wrap')]

def get_article_img_url(bsobj):
    return [img['src'] for img in bsobj.find_all('img', 'thumb api_get')]

def get_naver_news_article(url, news_bsobj):
    try:
        html = urlopen(url).read()
        soup = BeautifulSoup(html, "lxml")
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        return text
    except:
        return None

In [284]:
def make_url(query, sort, field, begin, end, page):
    url = "https://search.naver.com/search.naver?&where=news&query=" + parse.quote(query)
    url += "&sort=%i" % sort
    url += "&field=%i" % field
    url += "&ds=" + begin + "&de=" + end
    url += "&nso=so:r,p:"
    url += "from" + begin.replace(".", "") + "to" + end.replace(".", "")
    url += "&start=" + str(page)
    return url

In [285]:
def make_bsobj(url, delay=0.5, timeout=30, trial=10):
    ua = UserAgent(verify_ssl=False)
    count = 0

    while count < trial:
        try:
            time.sleep(delay + random.random())
            html = urlopen(Request(url=url, headers={'User-Agent': ua.random}), timeout=timeout)
            bsobj = BeautifulSoup(html, 'lxml')
            return bsobj
        except (URLError, SSLError, socket.timeout) as e:
            print('(Error)', e)
            print('reloading...')
            count += 1
            time.sleep(timeout)
    return None

In [286]:
url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%A0%9C%EC%A3%BC%EB%8F%84%EA%B4%80%EA%B4%91&sort=0&photo=0&field=1&pd=3&ds=2015.01.01&de=2015.01.30&cluster_rank=13&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20150101to20150130,a:t&start=11'
delay=0.5
timeout=30
bsobj = make_bsobj(url, delay, timeout, trial=10)
max_index = get_max_index(bsobj)
if max_index is None:
    print(1)

In [287]:
def get_arguments():
    # Argument configuration
    parser = argparse.ArgumentParser()
    parser.add_argument('--query', type=str, required=True, help='query to search on NAVER')
    parser.add_argument('--begin', type=str, required=True, help='crawling begin point (%%Y.%%m.%%d format)')
    parser.add_argument('--end', type=str, required=True, help='crawling end point (%%Y.%%m.%%d format)')
    parser.add_argument('--sort', type=int, default=0, help='search result sorting: 0(relevant), 1(newest), 2(oldest)')
    parser.add_argument('--field', type=int, default=1, help='search field: 0(all), 1(title)')
    return parser.parse_args()

In [288]:
def ran_num(n):
    ls = []
    num = random.randint(1, n)
    while n in ls :
        num = random.randint(1, n)
        ls.append(num)
    return num

In [289]:
df = pd.read_csv('news_data.csv', index_col=0)

In [290]:
df_e = crawl('엔씨소프트', f'./test.xlsx', f'2021.10.12', f'2021.06.06')

In [291]:
df_e

,naver_news_title,naver_news_articles,naver_news_img_url
0,"엔씨소프트 ‘리니지W’, 사전 캐릭터명 선점 열기 ‘후끈’",신규 월드 추가하고 6차 이벤트 진행 엔씨소프트(대표 김택진)의 신작 멀티플랫폼 M...,https://search.pstatic.net/common/?src=https%3...


In [292]:
df['start_date'] = list(map(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y.%m.%d'), df['start_date']))
df['end_date'] = list(map(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y.%m.%d'), df['end_date']))

In [281]:
news_title = []
news_article = []
news_img_url = []
for index, row in df.iterrows():
    df_e = crawl(row['stock'], f'./test.xlsx', row['start_date'], row['end_date'])
    news_title.append(df_e.iat[0,0])
    news_article.append(df_e.iat[0,1])
    news_img_url.append(df_e.iat[0,2])

(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found
(WARNING!) no results found


In [328]:
problem_data = pd.read_csv('./problem_data.csv', index_col=0)
problem_data['news_title'] = news_title
problem_data['news_article'] = news_article
problem_data['news_img_url'] = news_img_url

In [329]:
drop_index = problem_data[problem_data['news_title'] == ' '].index
problem_data.drop(drop_index, inplace=True)

In [330]:
problem_data = problem_data.reset_index(drop=True)

In [ ]:
problem_data = problem_data.drop(columns="start_date")
problem_data.to_csv('result.csv')

In [ ]:
tendencys = ['공격투자형', '적극투자형', '위험중립형', '안정추구형', '안정형']
tendency_explanations = ['시장평균 수익률을 훨씬 넘어서는 수준의 투자수익을 추구하며 이을 위해 자산가치의 변동에 따른 손실위험을 적극 수용, 투자자금 대부분을 주식, 주식형 펀드 또는 파생상품 등의 위험자산에 투자할 의향이 있는 유형', '투자원금의 보전보다는 위험을 감내하더라도 높은 수준의 투자수익 실현을 추구하고 투자 자금 상당부분을 주식, 주식형 펀드 또는 파생상품 등의 위험자산에 투자할 의향이 있는 유형', '투자자는 그에 상응하는 투자위험이 있음을 충분히 인식하고 있으며, 예ㆍ적금보다 높은 수익을 기대할 수 있다면 일정수준의 위험을 감수 할 수 있는 유형', '투자원금의 손실위험은 최소화 하고, 이자소득이나 배당소득 수준의 안정적인 투자를 목표로 함. 다만, 수익을 위해 단기적인 손실을 수용할 수 있으며, 예ㆍ적금보다 높은 수익을 위해 자산 중 일부를 변동성 높은 상품에 투자할 의향이 있는 유형', '예금 또는 적금 수준의 수익율을 기대하며, 투자원금에 손실이 발생하는 것을 원하지 않음']
tendencys_solution = ['']

In [333]:
tendency.loc[2]['tendency']

'위험중립형'

In [334]:
tendency = pd.DataFrame()
tendency['tendency'] = tendencys
tendency['tendency_explanation'] = tendency_explanations

In [335]:
tendency.to_csv('tendency_data.csv')